<a href="https://colab.research.google.com/github/sorayutmild/News-Headlines-Generation-Using-Transformers/blob/main/Generate_news_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Aug 25 07:12:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# https://drive.google.com/file/d/16IhVdoLuFso28TVIpYoIogco_LZg1e17/view?usp=sharing
!gdown --id 16IhVdoLuFso28TVIpYoIogco_LZg1e17

# https://drive.google.com/file/d/1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6/view?usp=sharing
!gdown --id 1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6

!unzip -q ./text_files.zip

In [3]:
import pandas as pd
import os
import re

In [4]:
df = pd.read_csv('sanook_news_all.csv')
df = df.drop_duplicates('Link').reset_index()

display(df.head())
df.info()

,index,Title,Link,Date,Views,File_name,Label
0,0,คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาที...,https://www.sanook.com/news/7722298/,5 เม.ย. 62,"1,925,802",การเมือง_1.txt,การเมือง
1,1,คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต.ค.,https://www.sanook.com/news/1148178/,16 ต.ค. 55,"1,074,542",การเมือง_2.txt,การเมือง
2,2,อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร,https://www.sanook.com/news/1596593/,22 พ.ค. 57,"788,755",การเมือง_3.txt,การเมือง
3,3,ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก,https://www.sanook.com/news/1773902/,2 เม.ย. 58,"736,582",การเมือง_4.txt,การเมือง
4,4,ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังปร...,https://www.sanook.com/news/7723326/,25 มี.ค. 62,"706,607",การเมือง_5.txt,การเมือง


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14858 entries, 0 to 14857
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      14858 non-null  int64 
 1   Title      14858 non-null  object
 2   Link       14858 non-null  object
 3   Date       14858 non-null  object
 4   Views      14858 non-null  object
 5   File_name  14858 non-null  object
 6   Label      14858 non-null  object
dtypes: int64(1), object(6)
memory usage: 812.7+ KB


## Text preprocessing

In [5]:
def read_text_file(text_path):
    with open(text_path) as f:
        texts = [line.strip() for line in f.readlines()]
    return texts

def get_raw_text(text_list:list):
    '''
    text_list : list of text paragraphs
    return : raw text
    '''
    return ' '.join(text_list)

def include_title(title, text_list):
    return [title] + text_list

def tokenize(text):
    tokenized_text = [word for word in word_tokenize(text) if word and not re.search(pattern=r"\s+", string=word)] # segment and not ' word '
    return tokenized_text

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_special_char(text):
    special_char = r"""[~@&#<>,}{()*:;"'-+=_?"\"/$%^%$!ๆ‘’“”…่่่่่่่่่ạễ•′–]"""
    text = re.sub(special_char, '', text)
    text = text.replace("[", '').replace("]", '').replace('-', '')
    return text

def clean_text(text):
    '''
    1. link
    2. symbols, numbers, special characters
    '''
    #pre rules
    text = text.lower().strip()
    text = replace_url(text)
    text = replace_special_char(text)
    cleaned_text = text

    return cleaned_text

In [6]:
text_file_folder = './text_files'

df['File_path'] = [os.path.join(text_file_folder, fn) for fn in df['File_name'].values]
df['list_text'] = df['File_path'].map(read_text_file)
# df['list_text'] = [include_title(t, lt) for t, lt in zip(df['Title'].values, df['list_text'].values)]
df['raw_text'] = df['list_text'].map(get_raw_text)
df['cleaned_text'] = df['raw_text'].map(clean_text)
# df['list_clean_text'] = df['cleaned_text'].map(get_list_text)
df.head()

,index,Title,Link,Date,Views,File_name,Label,File_path,list_text,raw_text,cleaned_text
0,0,คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาที...,https://www.sanook.com/news/7722298/,5 เม.ย. 62,"1,925,802",การเมือง_1.txt,การเมือง,./text_files/การเมือง_1.txt,[ผลการเลือกตั้ง 2562 จะรายงานสดหลังจากปิดหีบเล...,ผลการเลือกตั้ง 2562 จะรายงานสดหลังจากปิดหีบเลื...,ผลการเลือกตั้ง 2562 จะรายงานสดหลังจากปิดหีบเลื...
1,1,คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต.ค.,https://www.sanook.com/news/1148178/,16 ต.ค. 55,"1,074,542",การเมือง_2.txt,การเมือง,./text_files/การเมือง_2.txt,"[คอหวยฮือฮาป้ายทะเบียนใหม่รถนายกฯ ""รปภ.-ขรก.-ส...","คอหวยฮือฮาป้ายทะเบียนใหม่รถนายกฯ ""รปภ.-ขรก.-สื...",คอหวยฮือฮาป้ายทะเบียนใหมรถนายกฯ รปภ.ขรก.สือ แห...
2,2,อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร,https://www.sanook.com/news/1596593/,22 พ.ค. 57,"788,755",การเมือง_3.txt,การเมือง,./text_files/การเมือง_3.txt,"[รัฐประหาร, 23 พ.ค.2557, 15.45 น. บก.จร. แจ้งป...",รัฐประหาร 23 พ.ค.2557 15.45 น. บก.จร. แจ้งประช...,รัฐประหาร 23 พ.ค.2557 15.45 น. บก.จร. แจ้งประช...
3,3,ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก,https://www.sanook.com/news/1773902/,2 เม.ย. 58,"736,582",การเมือง_4.txt,การเมือง,./text_files/การเมือง_4.txt,[หลังจากมาตรา 44 ถูกนำมาใช้อย่างเป็นทางการ มาท...,หลังจากมาตรา 44 ถูกนำมาใช้อย่างเป็นทางการ มาทำ...,หลังจากมาตรา 44 ถูกนำมาใช้อยางเป็นทางการ มาทำค...
4,4,ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังปร...,https://www.sanook.com/news/7723326/,25 มี.ค. 62,"706,607",การเมือง_5.txt,การเมือง,./text_files/การเมือง_5.txt,[ผลการนับคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร...,ผลการนับคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร ...,ผลการนับคะแนนการเลือกตั้งสมาชิกสภาผู้แทนราษฎร ...


In [7]:
df['cleaned_text'][100]

'ผู้สือขาวรายงานวา 23 เม.ย. ทีศาลากลางน้ำ วัดเทพศิรินทราวาสราชวรวิหาร บรรยากาศการรดน้ำศพ นายบรรหาร ศิลปอาชา อดีตนายกรัฐมนตรี ทีถึงแกอนิจกรรม เมือชวงเช้าของวันที 23 เม.ย. ได้มีบุคคลสำคัญจากแวดวงการเมือง ทยอยเข้ารดน้ำศพเป็นจำนวนมาก ขณะที พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรีและหัวหน้าคณะรักษาความสงบแหงชาติ คสช. นายทักษิณ ชินวัตร น.ส.ยิงลักษณ์ ชินวัตร และ นายอภิสิทธิ์ เวชชาชีวะ อดีตนายกฯ ได้สงพวงหรีดดอกไม้รวมไว้อาลัย จากนั้นเวลา 17.00 น. พล.อ.เปรม ติณสูลานนท์ ประธานองคมนตรีและรัฐบุรุษ เดินทางถึงศาลากลางน้ำ เพือเป็นประธานในพิธีพระราชทานน้ำหลวงอาบศพ โดยกอนเริมพิธี พล.อ.เปรม ได้พูดคุยกับคุณหญิงแจมใสและครอบครัว โดย พล.อ.เปรม ได้วางมือแตะไปทีแขนของคุณหญิงแจมใสเบา เพือให้กำลังใจด้วย โดยคุณหญิงแจมใส ศิลปอาชา พร้อมบุตรทั้ง 3 กัญจนา วราวุธ ปาริชาติ และครอบครัวศิลปอาชา ได้กราบขอขมาศพนายบรรหารเป็นครั้งสุดท้าย กอนประกอบพิธีบรรจุลงโลง ซึงบรรยากาศเต็มไปด้วยความอาลัยรัก ทั้งนี้ จะมีพระพิธีธรรม สวดพระอภิธรรมในพระบรมราชานุเคราะห์ 7 คืน ตั้งแตคืนนี้จนถึงวันที 29 เม.ย. 2559 ณ ศาลา 11 กวีนิรมิต วัดเทพศิรินทร

## Modelling